In [1]:
# Crawling에 필요한 module 선언
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# (201105) 추가한 모듈
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

# lxml
from lxml import html
import xml.etree.ElementTree as ET

# pandas
import pandas as pd

# requests, json, re
import requests
import json
import re

In [2]:
# 유튜브 정식 카테고리 k, v dictionary
youtube_category_dict = {
    'Film & Animation' : 1,
    'Autos & Vehicles' : 2,
    'Music' : 10,
    'Pets & Animals' : 15,
    'Sports' : 17,
    'Short Movies' : 18,
    'Travel & Events' : 19,
    'Gaming' : 20,
    'Videoblogging' : 21,
    'People & Blogs' : 22,
    'Comedy' : 23,
    'Entertainment' : 24,
    'News & Politics' : 25,
    'Howto & Style' : 26,
    'Education' : 27,
    'Science & Technology' : 28,
    'Movies' : 30,
    'Anime/Animation' : 31,
    'Action/Adventure' : 32,
    'Classics' : 33,
    'Documentary' : 35,
    'Drama' : 36,
    'Family' : 37,
    'Foreign' : 38,
    'Horror' : 39,
    'Sci-Fi/Fantasy' : 40,
    'Thriller' : 41,
    'Shorts' : 42,
    'Shows' : 43,
    'Trailers' :44
}

# 샘플 키워드 리스트
sports_list = ['한국 프로야구', '해외축구', '바르셀로나', '배구', '손흥민']
movie_list = ['영화리뷰', '영화 명장면', '명대사', '한국 영화']
kids_list = ['뽀로로', '헤이지니', '카봇', '보람튜브', '핑크퐁', '공룡']
music_list = ['인기 음악', 'k-pop', '팝송', '카페에서 듣기좋은 노래', '호른', '클래식', '째즈']
comedy_list = ['런닝맨', '1박2일 다시보기', '무한도전', '놀면 뭐하니', 'tvN']
game_list = ['롤', '피파', 'fps', '배그', '서든어택', '어몽어스']
politics_list = ['바이든', '트럼프', '대통령', '중국']
cook_list = ['백종원', '자취음식', '편의점 음식만들기', '몸에 좋은 음식']
animal_list = ['고양이 영상', '비글', '해수어', '상어']

# 모든 키워드를 다 합친 리스트
all_search_keyword_list = sports_list + movie_list + kids_list + music_list + comedy_list + game_list + politics_list + cook_list + animal_list

print("All keywords list: ", all_search_keyword_list)

# video_id_list
# sports_video_id_list = []
# movie_video_id_list = []
# kids_video_id_list = []
# music_video_id_list = []
# comedy_video_id_list = []
# game_video_id_list = []
# politics_video_id_list = []
# cook_video_id_list = []
# animal_video_id_list = []

# 샘플 키워드 딕셔너리
# search_dict = {
#     'Sports' : [sports_list, sports_video_id_list]
#     'Movie' : [movie_list, movie_video_id_list]
#     'Kids' : [kids_list, kids_video_id_list]
#     'Music' : [music_list, music_video_id_list]
#     'Comedy' : [comedy_list, comedy_video_id_list]
#     'Game' : [game_list, game_video_id_list]
#     'Politics' : [politics_list, politics_video_id_list]
#     'Cook' : [cook_list, cook_video_id_list]
#     'Animal' : [animal_list, animal_video_id_list]
# }

All keywords list:  ['한국 프로야구', '해외축구', '바르셀로나', '배구', '손흥민', '영화리뷰', '영화 명장면', '명대사', '한국 영화', '뽀로로', '헤이지니', '카봇', '보람튜브', '핑크퐁', '공룡', '인기 음악', 'k-pop', '팝송', '카페에서 듣기좋은 노래', '호른', '클래식', '째즈', '런닝맨', '1박2일 다시보기', '무한도전', '놀면 뭐하니', 'tvN', '롤', '피파', 'fps', '배그', '서든어택', '어몽어스', '바이든', '트럼프', '대통령', '중국', '백종원', '자취음식', '편의점 음식만들기', '몸에 좋은 음식', '고양이 영상', '비글', '해수어', '상어']


In [ ]:
# 샘플 키워드 리스트 #2

In [6]:
# 검색어 결과별로 비디오 id를 모아주는 함수 정의
# 리턴은 dictionary
def get_search_keyword_scrap_video_id(search_keyword_list):
    # 최종적으로 리턴할 dictionary 정의
    youtube_id_dict = {}

    for word in search_keyword_list:
        # youtube_id_list
        youtube_id_list = []
        
        # webdriver define
        driver_path = './chromedriver'
        driver = webdriver.Chrome(driver_path)
        driver.implicitly_wait(30) # or bigger second
        
        # 열고자 하는 search url 정의
        url = "https://www.youtube.com/results?search_query=" + word
        print("full url: ", url)

        # 해당 url로 창 열기
        driver.get(url)
        time.sleep(5)

        # 첫 동영상 시작 시 시작번호(index = 1)
        index = 1
        SCROLL_PAUSE_TIME = 2

        # infinite scroll
        while True:
            try:
                youtube_video_id_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/ytd-video-renderer['+str(index)+']/div[1]/div/div[1]/div/h3/a'

                # 해당 페이지의 8개의 영상을 모두 확인했다면 그다음으로 가기위해 스크롤
                if index % 4 == 0:
                    # 2초 대기 (대기 시간은 크롤러의 정밀도에 따라 조정 가능)
                    # driver.execute_script("window.scrollBy(0, 1080);")
                    # time.sleep(SCROLL_PAUSE_TIME)
                    # driver.execute_script("window.scrollBy(0, 1080);")
                    # time.sleep(SCROLL_PAUSE_TIME)
                    # driver.execute_script("window.scrollBy(0, 1080);")
                    # time.sleep(SCROLL_PAUSE_TIME)
                    last_height = driver.execute_script("return document.body.scrollHeight")

                    while True:
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(SCROLL_PAUSE_TIME)

                        new_height = driver.execute_script("return document.body.scrollHeight")

                        if new_height == last_height:
                            break

                        last_height = new_height


                youtube_title_link = driver.find_element_by_xpath(youtube_video_id_xpath)
                youtube_id = youtube_title_link.get_attribute('href')
                youtube_id = youtube_id.split("v=")[1].split("&")[0]
                youtube_id_list.append(youtube_id)

                # 리스트에 들어간 데이터 중간 확인
                print("index_no: ", index)
                print("youtube_id: ", youtube_id)

                index += 1

            except Exception as e:
                print()
                print("Exception: ", e)
                break
        # dictionary에 검색 키워드 별 리스트 추가
        youtube_id_dict[word] = youtube_id_list
        # youtube_id 수집이 끝난 브라우저 닫기        
        driver.close()
    
    return youtube_id_dict

In [3]:
youtube_id_dict = get_search_keyword_scrap_video_id(all_search_keyword_list)
print("youtube_id_dict: ", youtube_id_dict)

NameError: name 'get_search_keyword_scrap_video_id' is not defined

In [7]:
# youtube_id_dict
youtube_id_dict = {'한국 프로야구': ['doorlUyE9pI', 'L0Y38pQbEeQ', 'VjwJSj0AW4g', 'A5vCPtjOe8Q', 'Zeja7fC1Okc', '7ARbpqOgHM8', 'WC2MB_lZ0HQ', 'ugsFwK74KDU', 'FeqhJVzlE-4', 'B-ZFW6-FBJ8', 'kH-ht4CyUKg', 'kQxq3PNE0gw', 'leBGyyECGPk', '7eYu507yrQ8', '-P68flOonvE', 'FCEPBiUPPho', 'pp0a2SbPwRA', 'VFufiK95fGU', 'De58PXoH4IU', 'wVo7-0zDNps'], '해외축구': ['zP-gjvYibTM', 'DWwbxsVwPEs', 'jKJkhkU_Uy0', 'TI5LZt1MzkA', 'nxGNyWF2ASQ', 'XmDrZ_2Kg2U', 'Gf9t-ySPfao', 'BdHFW-U6_Pw', 'HuXhzWuRnos', 'VpEre3ksO5w', 'lchK_-AqmaQ', '5caHF8J62G4', 'epxwH4xMq9Q', 'N-gP5KyrWSU', '8FuIAt0Cz5M', 'LKJwO9PRuBg', 'qUTqVD8pMpI', 'b_y5cIw8toQ', '3Z0N2jTS3_k'], '바르셀로나': ['iwt-dKlsyc8', 'Ww2f6hcbgPo', 'DWwbxsVwPEs', '1R6eqSmlyKs', 'QBm8vthloeQ', '92oRB3yYSi4', 'V_1SvHgRQ9E', 'AQ2p8hxX90Y', 'LKjkugAHCmY', 'xChIWLtcT6Q', '2d6JJDZCIvc', 'TOC126WWIgc', 'xpHO-0JpP6w', 'b2NVhLUTLdM', 'j8onFrjLUdA', 'PtfS4o9o_gE', 'pkg__2CzXSk', 'omHQiFoD2JU', 'aHwfftlTK2c', 'Fw5AJryeTB4'], '배구': ['hDjlMQwI3n8', 'qSVzA_su0_Y', 'XE9WwgCZVic', 't4rmTfHRe58', 'zJBhOTezumM', '2GpNMDhpXPg', 'VT6lZWcQrQ8', 'I4nmh6PFF0Q', '6EJhGux0_g0', 'H_GrMKiRPLI', 'lpwR7eJS6OA', 'rOhAG74chwA', '_5ERlIGgWtA', 'ceS72R7gRAo', 'Szozl8dYPR8', 'xbJCrxfXndI', 'tsyM11aPQj4', 'SFRlCcDqZJY', '8f7c28DmNDY', 'RQRVAxBNsWg'], '손흥민': ['kd_YLeWRltY', 'XZzE7jO51_M', 'y7Ywwm8bvMA', 'UgrU6pDCJRU', 'omcGbZ5RbzI', 'F5b8sr4BKIg', 'iShyDckElHY', 'ATo-kwX0sJc', 'nPCeN8khVDA', 'Ka_vHYMOjlc', 'iQ1oi_p7ZCg', '7pwRhZnA8tk', 'dpyHS7vMNBA', 'rRF8v0v6m-o', 'na0xf3J0v4w', 'JZw1jooO4HA', 'GU3qCbMCEgI', 'ahQobPP5bF0', 'Icxcxeb8Yxg', 'JHp-ooo75C4'], '영화리뷰': ['A-Xu6CV-eRY', '0nHoVO_5Ius', 'fnCGDkR3t-A', 'cG2U9rFSKO4', 'KEQNpEOC660', 'j_-YWwVncQQ', 'AA0Z8yROvYg', 'CLveKMZ5R9E', 'x26o2y0ytEM', 'sxNkEbbDAbo', '1nY-8ZEngx0', 'WT9sN4hkNsI', 'to0yQ-8q-as', 'M54Z50Bcud8', 'bAdFEaLVaa8', 'aNVAxFANHlo', 'vgY--6eJj7s', 'ITPU2bNc4ts', 'kSDZ6KjgkAE', 'FkFkQFQkF84'], '영화 명장면': ['hdU_5M9U9tk', 'ojttyD8aMR0', '12d5V0lzdYg', 'u1VezOU7bt4', 'nUxfxGYjbO4', 'hd6bGMJ-Lwk', 'l6GGHe2hgtk', 'UvIEBsJnQyk', 'M6fa1SWeya4', 'um3c0hMzz5M', 'g3XqWtn681Y', 'jS5-1KUapuU', 'gb2s1eF_EGg', '7wWHZYS7JV4', 'VVGCzsm9r4E', '1gZElKODBBc', 'Joa8f3IN4dQ', '1N_3Du-JBIo', 'bxZXN3zcE5I', '_EWIe0PjQ6g'], '명대사': ['ojttyD8aMR0', 'hdU_5M9U9tk', '0ZHqB7Fplu0', 'goGst1xKBQ4', 'LJO7-pN9hAk', 'EwERszcjwaE', 'F4SUPzLdbu0', 'zm8Ts0XoErg', 'y58EHjtKnBA', 'Q9oat_q3UqQ', '9fCqGqqrNVk', 'SQfv__xer3A', 'OQVzaW0oBgE', 'XJkG4Kiykm8', 'pYCtl0CEv5w', 'B2mOlO1ERtw', '_ZWKRJY5Ghs', '8nRNGweY46g', 'R5NWIGAhHYA', 'dxusUW9_xZE'], '한국 영화': [], '뽀로로': ['xmHdqnDfqg4', 'xlmPbjAcRXQ', 'xCMPnwDkfDI', 'mJfrwP4wufM', 'sZkRT09tXEc', '_ee83gRY5qo', 'fDjsD8atx1U', 'Idc7hWtOeL0', 'ed56y-G4FcI', 'FfC8ZWAqV5Q', '02V4I_73FcE', '65ejhvVjViI', 'TtAsS3Sjq64', 'ZHSQTVB8zOI', 'g9kQEPDN5PU', 'XFUi4KTlkgA', 'UG4Z1edn8bQ', '-O9hpaEK_-Y', 'rCUXGWk3DHg'], '헤이지니': ['Ud7appVhiC0', 'VNMj3dgzMHw', 'pi82H6eZRSU', 'AKT2Z_cE5t0', 'olLFXLKEzvc', 'qX6odkUJ1lg', 'alQaHRb3HDk', 'xQfqaTTD1JY', 'pN4WmaIQQBU', 'PgcQ81tRV4w', '8zqrI3za7LM', 'a6p_TWoeGH8', '9Ks-DP_wp1s', 'MOMEssNSA8o', 'a-rysSCYT9w', 'IGCyavURxMg', 'Vq7jlQ0xaks', 'CN9hMZLphUU', 'OE-S7rEBXwk'], '카봇': ['fMRARhSQILg', '1qP9N175okQ', '2enS5vTUsWw', '3BEUwnqzQnY', '1WzYvNTR0Zk', 'mQfCs7KIjXs', 'tdYxvPfVyGY', 'BS28qiDPRWI', 'cfjTExuA_nM', 'PjE-4QdKufY', 'OJeRcK1Ml3k', 'zglQKs1adV8', 'xdJ4WcBh4Vg', 'kPL0gXwwwjE', 'al6v7SU3NNg', 'uk-NRDRiZaA', 'bHJ1SEarxtU', 'VXVmF1vnnFw', '0Qmt9bLbBOg'], '보람튜브': ['bzQIbntYh7c', 'X2Zmf3c0ccA', 'MqQUUdxrzAQ', 'NPTTJqcPhXU', 'PZYgwhJB7Oo', 'xEpL0_JjpHw', 'ht42dD1erJk', 'L__85-J9NXs', 'dumlv30AHJU', '3Xun01p1oZI', 'UpDo2vfusGw', 'hWdm8cHd7LU', 'YFe_L83o38w', '5Fw4eeh8Qys', 'uawQagHG5js', 'iQiGeK2vFpo', '4AC42XrAuoo', 'sC2AjRB7kWU'], '핑크퐁': ['WEHld_90DlM', 'dBD54EZIrZo', 'dkzNqzIC2-k', '_ELRHM_Fc14', 'kw_zTVuatk4', 'Z6bVY6Q87xY', 'hvTq3bLY2N8', 'rwnqNUALVbs', 'oqQyELG6K8Y', '9XTPX-t1TqM', '0rCMJkLh9VA', '6bbMFMRei2I', 'cq_JzIjis40', 'EeVRIya_pfA', 'wVlP86rgSQM', 'tTIw6PVxy5c', 'hg_HM447sOU', '8_7BZHfSmfM'], '공룡': ['JLFtlXPuFaw', '7IadsIPjO9I', 'v2tsHXQI6d0', 'p_Fj6KmgXX4', 'IRZF0IZ6BhI', 'vvAly7LI23U', 'JAYa0ffVVQY', 'm40Dt_TZFBI', 'IUAjhm7oE9c', '-FqABXx2t4o', '18N7NHp9yS4', 'iz2Pd_eP7Hw', 'Qj45k7hCIwI', 'xcFaoRPFT1A', 'f2JwkdOlBVU', 'RnO-GXbajww', '1wqQJpXBL_8', 'H9WA-pTdFOw', 'L0qoyEhwleU'], '인기 음악': ['zjGzPiYGhhY', 'khu6-Cdn8E4', '848a1kMwRpE', '_BVqb9vWZ_k', 'wMSh6wFNtuo', 'Rum2524qZPU', 'B8FyB1vf5lY', 'YyeylfmSVP8', 'vpbR3NEJmJM', 'Z2_ygVUxUcc', 'USgbeoynpw4', '0lI5xI_gIIs', 'u-E92ep1BDg', '-kkyXf_D_TE', 'yJuLOl1V-lQ', 'iHfqD2RYT2I', 'b5rrGmpZlDY', 'JzziwpDrigs', 'N5Nq3xBs5U8'], 'k-pop': ['rLxLVeN7sgc', 'GWFfWbDw5Q8', 'ZNV-pnhHdEs', '0nxsS4B85E8', 'ZeerrnuLi5E', '7c4gAOZ1JAw', 'qM2NJNrJDQI', 'zWiVhjmM9Hs', 'dH8e3Q_ExWk', 'T56iHifXrsY', 'h9TmtANeOmY', 'kOzoFmkXxZE', 'bgVixCyg7fA', 'Z3D8bN0UH_8', 'CM4CkVFmTds', 'UOxkGD8qRB4', '446ZxblW6pU', 'elPoQ_M4gaA', '0QbVt7N9FNo'], '팝송': ['_kd8bp0pp8U', 'HMfDUdNVjcw', 'NnSAsgkTgtY', 'mxJGDa7ThbE'], '카페에서 듣기좋은 노래': ['SUEbgBbJ1BQ', 'e9AkF2S1BR8', 'VcIs0Lvbsj4', 'SORD03t7nlo'], '호른': ['lNuJVfe-t3o', '8NdzRGJ4xJo', 'K10vpdVZ58Q', '4nU1Ny_Wplw', '6AVa7gU0v7E', 'RNo5l8iMRVw', '5WN20BcJsgw', 'fhT9f0ZKIRg', 'QB74ZZJk_bk', 'KKI51KIGvVw', 'CgNJSfwxiYw', '5fro7jmT3_M', 'MCH_MDdVEfM', 'iyEhtV-OSjw', 'TuZ3-O1EUIc', 'xAJQpZT5Q2g', 'M3OFiQtRI_A', 'SIoBIxKGL-Y', 'EY11eklYA5s'], '클래식': ['qsP9C-knpGY', 'vSTXLYojkiA', 'MEHF1MU3qyw', 'y_1IOZB-_mI'], '째즈': ['VcIs0Lvbsj4', 'MxWdsVMFm80', 'ENPm0YrHyVc', 'drQY0hZ3fGU'], '런닝맨': ['5yZAUy8cgqs', 'rYTNyMwdlWg', 'c-KLSqKrIf0', 'Oy3vm-DHzeI', 'bVxUuqsP1Qg', 'MY7Q1BHiMZk', '8um_YMOSHqI', 'd2QklvGTJVg', '6e1YGd3NH-w', 'GwtSzxIBnag', 'SaK5gtS8sLg', 'oAgFrhL_YII', 'B7VCUMyTPyc', 'KjPzlC6QE0Y', 'DceGdbWedKk', '61DVaMD7WXQ', 'fvDBOCrOL4E', 'Auw-3xY1waI', 'iXLXLCcGINw'], '1박2일 다시보기': ['10anaq7oTp0'], '무한도전': ['_MnoMA_Y7C4', '-uvhZ7o3x04', 'wFDXalRMpXw', 'A1EmZJDFm6k'], '놀면 뭐하니': ['YAgaicXOjas', 'PtvKVNV-4to'], 'tvN': ['hc5petZeyio', 'wLCmXq2_yRo'], '롤': ['lZX6rR2hxmU', 'X96TunaR3Bo', '87M8if6zrrU', 'sKQIFWIXyMY', 'oaIV0s5DXXI', '0ayv0_ujL0k', '1PqhVpGai1c', 'ErhShv4ELL4', 'w_VbC-krBM0', '1ArtO40izV0', 'W-4f0j-CVf4', 'MugUuG4RkL0', 'cmw0vTjjEtg', 'jTwN9qNJ2Zo', 'CCuCAUMuQKA', 'V_t76T4z3LU', 'ZfITPOIHrH4', 'mMK0Ihar6Ds', 'T8BgIPb67vI', 'ev0bEDV3RCo'], '피파': ['9SBPATOqRp4', 'zqxPHdnRo3M', 'le21IGkvvmU'], 'fps': ['6R_3gwH7rJ4', 'PyI4i_Pcl5Q', 'cCyV2qNQkGs', 'lndIkNjkMq4', 'UaCk6fH8rD8', 'zLnup1gskzU', 'oFNsLSN8rZU', 'SdQL9_zfD0E', '5b1r0Sa58Ko', 'L0L8wg_GKl0', 'OBT53lVZJrY', 'PpNSKdrweJs', 'SI81QuBE28M', 'w5kQU8NvIfk', '09H9O_dnXgY', 'VNtiWnCnvcU', 'lrTp0EbRpxE', 'iJ4juLEh088', 'AM5gEsqIrQk'], '배그': ['mZOGvvG8xCo', '2VdWU9XLFBs', 'Tbv62o0b664', 'cj0Em8Gcioo', 'ZHNTA57Tngo', '-LYzPhPnON4', 'nK81lcIgDWY', 'S8XzepC1JkU', 'Fxnivln-TDM', 'vgbMapTQoD4', 'TExCkw_z2II', '8MoHqR9YTVE', '8CbZN4iFYUQ', 'l5PgztHt0ms', 'ZJB4eK6_8IQ', '25AYCGJZns8', 'o_0qP7Ilcy8', 'hQYIxyhvh-4', 'II5hwXKdpHU', '97n9hruy6hY'], '서든어택': ['nFSzBK0vV74', 'qSUBRcIGkdg', 'oj2orHOag7o', '1Qsa0WwLBr4', 'p3xanvMjssc', 'lKZZjK40tEw', 'rOAxMrUxQSU', 'QfZ0gZ6ccJ0', 'm9VDesk6k_A', '5uFec_F3TvY', '4VUazdjOQZ0', '-4AmtlKYAkA', 'S-Vt_ffFqhM', 'nDGSxrrtuUg', 'luOHdx6loZI', 'ipsBOH_LIk4', 'p7_DOMqh9kQ', 'upEiKDm4x1g', '83-w6J84eWE', 'WIZKq5kRGg0'], '어몽어스': ['O5SnsZDns-c', 'EdpKwOG2X80', 'ywLJOsjckFQ', 'vzw_BAb4etc', 'lvsvSQMHxmg', '_Rrav6oDSXA', 'IqoXggsjFfQ', 'GF_UtlAzbr0', 'zLf0zMjZJTM', 'AA9EDlKhAyE', '0yc2MH7kyZk', 'T15XnTl2skk', '3ZaeB5-mQt4', 'oKNEHc1xiQ8', 'TapEl3bKBu8', 'MQyDvGysSlY', 'nt67Er2a87g', 'n5WOw4Ep4mY', '-PxvczUNEnw', 'WruznbP0DhE'], '바이든': ['s7QhC3exrdo', 'LQElZ4N89Hk', '9-_FOYbW--U', 'MS7zuIWmu0Q', 'vK9HzPtdafw', 'XdTqtNjX4PA', 'ybl4AHLTVIE', '750xpOHhSnI', 'pVTRbZn10_c', 'oJy5dczEmuE', 'lPazD0wp1zs', 'g-KxX5XGu5U', '2WAgoPunn5Q', 'm4axFZeUyD4', 'IkX-8YHofjc', 'BNBM_daPZtI', 'aRju707_ymQ', 'jPGIdCB29HQ', 'WRUUEusffYQ', 'pT18Ld2J-EM'], '트럼프': ['AaP8cpFA6Qg', 'WY3qSffhR6o', 'QX-1ZY3R2YY', 'f_5UCufu9xc', 'e96IeaSYC5I', '9-_FOYbW--U', '376iFtKv-7o', 'z1iXjQDwLxU', '750xpOHhSnI', 'vg_LETfsHu4', 'MS7zuIWmu0Q', '5Cv7jPp2_iM', 'Y3nxMnZ42b8', 'sYd7qtzzyaQ', 'hgY5kndWD1M', 'nH8bFg6ijqE', 'QL6dTGwUkzI', 'LQElZ4N89Hk', 'g-KxX5XGu5U', 'D3OReBCp8eY'], '대통령': ['91xMG7oFE1M', 'f_5UCufu9xc', 'ATkS3GElG5g', 'cS_5ESUcZAA', 'zuU_9FAaaFI', 'YRzSPwV9LIE', 'k2XNiN91J_A', 'Z7zD7ajIUX4', '9_PgIIk6h80', 'jDDkqDMT0Gg', 'zyyCN1uuGeQ', 'iLHWeMLv6-U', 'faCoL2kptPU', 'jKIS0U6pB3k', 'Gu7kxLD_MuM', '_aNq1HxWB1Y', '4A3P9Tgm8wc', 'oWxBSKIm8JM', 'bF2v8vbAG1I', 'yY8bWEqUw78'], '중국': ['tdulZOowSz8', 'td6YWqnbpbA', 'ihCYHrJwpVY', 'bZvPPq3VhcI', '2fWjdD2VCjY', '76e50fnjC3w', 'YPi6ne6D64Q', '3T3R7ItyMow', 'iu9ckQCmU3k', 'iOLB9HFuXcA', 'tZNELPaXF1c', 'h_KV_YR2_OU', 'Gwb-PKAo9io', '2crtBfyGELo', '1WzEFhg7K3k', 'tyfqZg3WJ4g', '-0ydCEcxQ-c', 'MDK5WvVAwgM', 'blU7Mxv8RaY', 'DxakZWazMws'], '백종원': ['qWbHSOplcvY', 'm0eeDwbhzM8', 'kR77WlHRZrs', 'iGhOobaQHso', 'wpUsg7f-F4o', '6e-IbuuD6ZU', 'hc5petZeyio', 'wHuvJKdu2TE', 'udirCMIxs_c', 'ZNOExo_FD3E', 'uF_2nYy74lw', 'fEYJSPSGaLI', '0bnFoRQebq0', 'kfGi3rTct8c', 'Mraq0yBPQXE', 'mwMYLipQluk', '-D1qfgYuBfw', 'GP8_7D5eM6A', 'KhKThbfYoMs'], '자취음식': ['rEnnfMFAijU', 'AcWqE4JC9Pg', 'kmIYGGlua9s', 'F6k4ea_CljM', 'uJq4fSwRbD0', '5j-q3gL1gZE', 'ywuRIlq8dBA', 'ePYy2if0lMY', 'PAh36xoHqg0', 'WvYF1Ax1G7c', 'kR77WlHRZrs', 'deg7pjD0lOs', 'poVEutWwRCo', 'g0rJjcPiWnI', '-4aR8F_T7rU', '_wQvrd4gCiU', 'wevmmDP5jgw', 'Jd9af-re9iE', 'SlRaNQardN8'], '편의점 음식만들기': ['hnvIoHkKKSc', 'TKFpBXl2w-A', 'z6VAEVhluDs', '3iMq4Kdq8cQ', '5AJzx-6sQ5M', 'lgRXx7DJbCY', '-hyL57AtnX8', 'Tcy74iZYNSM', 'U75Zto-R_cY', '6BOOefgnarE', 'Oe2n-UzTuZ0', 'o0QxvVlY7pU', 'g_yyvAKHpsk', 'lv6R8wGjLG4', 'mdv3cIgE2r8', 'BDe5vQfc0iw', 'lO0V19pWE_s', 'OgjUBxTS1FE', 'beZrxBOOWtc', 'xJCoByOe0D8'], '몸에 좋은 음식': ['sO0Vk4v84-Q', 'QASWk6Zslwo', 'ofxYXCQUIZU', '69rSSAW-sGs', 'QBhBvGu0LW0', 'WsinkRH16UU', '6grBm64NUuA', 'fumpEZHiGSs', 'YTGmipgUazY', 'Y_vs2J-t-Kg', '7t6HR9fNkus', 'FU2h84mSLxM', 'Vok2Zqq8SH8', 'xI6Q-I4wxdM', 'WaPeTq1mg-s', 'fxMZRqFU3us', '3zZ_mrXPPNk', 'ru9bXwn6_Eg', 'NzI0TEyZPCA', 'yn2ILPliDfI'], '고양이 영상': ['1R752fMKjvA', 'f2Wcy5mi9Ck', 'I1ms7iiGgYQ', 'lZJFhEStBS8', 't9P4k27dogk', 'nWMMbZG9aKM', 'OWOPZApMOqE', 'rbz5siyFS9s', 'uZFdYhz4ZFU', 'bwQ_jw4lZ3g', 'D99LZeZbdyk', 'QEZzngmPzlQ', 'LGv38Ny4RKc', 'G8Szvw2nGa4', 'RM4rK3vHa_c', '435bSRyWoAo', 'D52kRtlntGI', '2vBU7H6OZgw', 'QRFqphsEZeU', 'J3vLTifqG7U'], '비글': ['J_M-zwOJb2k', '95Tu-WpkGkQ', 'lE2hZXmHGDU', 'GClkqj5JXqc', 'lX7_I5IDQVw', 'K1eWhsxcPX0', 'gjdQkQkX7ks', 'x1TMRJUxxiY', 'Z9sZbnA6EyE', 'Dr-CcGrXb1Y', 'IU9hZ7yhMSM', 'FdP7CGs13I8', '3dRk6ZixmlY', 'gMBadP72Zx0', 'zwXqAo3hmzA', 'ojcj2LmdG3o', 'NvXG08eBtmI', 'oqkB6cVJ_-M', '2kEa4hTSJ7Q', 'iBoH-2-Oi3Q'], '해수어': ['2cxftQrBDHU', 'YxNDWhnzk0E', 'eb2nyvK6h2k', '9G5UqTVA7YI', 'QAhayrF5s1w', 'ecu8Ena_NDQ', 'fbSuRN23Dic', 'pfCbyzyl9B4', 'iP2lKfPJ6qE', 'dn99jOoZNJI', 'lV9hFbmD-PU', 'RieJTVpeEV0', 'MNzB5L60ytA', '73p6P0Xa0vU', 'jYEj3f3nncU', '4S1P0xe9siI', 'lvpQI9fdEEE', 'ZCFTEKR1Ypg', 'Nylo-SpEnFY'], '상어': ['zxWlDeaq2vY', 'cK8SLPmyJkE', 'n_Bs5Oht0ww', 'niqDv_Cho20', '4QNdcz1927s', '9jCOZWu7I-o', 'eKuetncvKLQ', 'k_DrienZyp8', 'Oxmo4Z8ydIQ', 'zcsJQvbgE5E', 's_3vVYHoilI', 'tbJmQ5zYmro', 'Jq2LqzNPhOo', 'n7DGG8_gBKE', 'lpaaUMluenI', 'EbPIhs0tfkU', 'LzLvCAR6M1w', 'ZtU3OU8DuJc', 'LQy6gAogliE', 'IMdowhiALuY']}

In [27]:
# 특정 검색어 키워드 기반으로 크롤링 해 주는 크롤러 정의
def scrap_search_detail_video_data(video_id_dict):
    # 모든 결과를 누적해서 보여주는 df 정의
#     youtube_raw_data_columns = ['title', 'authorNick', 'description', 'urlPath', 'thumbUrl', 'likeCnt', 'dislikeCnt',
#                                 'followCnt', 'viewCnt', 'duration', 'utbCategory', 'utbCateSN', 'isFamilyFriendly',
#                                 'titleHashTags', 'descHashTags', 'utbVideoID', 'authorID', 'yUploadDateTime', 'keywords']
    
    # 모든 DF를 누적할 DataFrame정의
    # total_df = pd.DataFrame(columns = youtube_raw_data_columns)
    
    # 각 video_id에 해당하는 유튜브 영상 별로 상세 데이터 수집
    for keyword, youtube_id_list in video_id_dict.items():
        
        print("==========outer loop==========")
        print("keyword: ", keyword)
        print("video_id_list: ", youtube_id_list)
        
        # 수집할 데이터의 리스트
        title_list = []
        author_list = []
        description_list = []
        url_list = []
        thumb_url_list = []
        like_cnt_list = []
        dislike_cnt_list = []
        follow_cnt_list = []
        view_cnt_list = []
        duration_list = []
        youtube_category_list = []
        youtube_category_sn_list = []
        is_family_friendly_list = []
        title_hash_tags_list = []
        description_hash_tags_list = []
        video_id_list = [] 
        author_id_list = []
        yupload_time_list = []
        keywords_list = []
        
        # youtube_raw_data를 받을 dictionary 정의
        youtube_raw_data = {}
        
        for video_id in youtube_id_list:
            print("==========inner loop==========")
            
            # webdriver 초기화
            driver_path = './chromedriver'
            driver = webdriver.Chrome(driver_path)
            driver.implicitly_wait(100) # or bigger second

            # origin_url 정의
            origin_url = 'https://www.youtube.com/watch?v='

            # 넣어주는 url 값
            url = origin_url + video_id

            # scriptTag에서 긁어지는 데이터
            script_tag_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[1]/ytd-player-microformat-renderer/script'

            # meta tag에서 긁어지는 데이터 -> soup.find, soup.findAll 이용해서 긁기
            request = requests.get(url)
            soup = BeautifulSoup(request.text, 'lxml')

            # ytInitialData에서 긁어지는 데이터 -> 복잡한 계층의 json 데이터 기반
            yt_initial_data_xpath = '/html/body/script[19]'
            
            driver.get(url)
            time.sleep(5)
            
            # 데이터를 담을 변수 정의(scriptTag, metadata, ytInitialData)
            try:
                # Data 크롤링
                # 각 유형별로 다른 로직으로 데이터를 크롤링한다
                # script tag id=scriptTag의 내부 HTML 내용을 가져온다(scriptTag data crawling)
                script_tag_str = driver.find_element_by_xpath(script_tag_xpath).get_attribute("innerHTML")

                script_tag_json = json.loads(script_tag_str)

                time.sleep(5)

                # print("script_tag_json: ", script_tag_json)
                # print("script_tag_json_type: ", type(script_tag_json))

                # Data result test
                title = script_tag_json['name']
                author = script_tag_json['author']
                description = script_tag_json['description']
                thumb_url = script_tag_json['thumbnailUrl'][0]
                view_cnt = int(script_tag_json['interactionCount'])
                duration = int(script_tag_json['duration'][2:len(script_tag_json['duration'])-1].replace(",", ""))
                youtube_category = script_tag_json['genre']
                # addition : youtube_category_sn
                youtube_category_sn = youtube_category_dict[youtube_category]

                print()
                print("=" * 20, "Youtube Raw Data Descriptions", "=" * 20)
                print("title: ", title)
                print("author: ", author)
                print("description: ", description)
                print("thumb_url: ", thumb_url)
                print("view_cnt: ", view_cnt, " type: ", type(view_cnt))
                print("duration: ", duration, " type: ", type(duration))
                print("youtube_category: ", youtube_category)
                print("youtube_category_id: ", youtube_category_sn)

                # meta tag data crawling
                is_family_friendly = soup.find("meta", {"itemprop" : "isFamilyFriendly"})['content']
                author_id = soup.find("meta", {"itemprop" : "channelId"})['content']
                yupload_time = soup.find("meta", {"itemprop" : "uploadDate"})['content']
                keywords = ",".join([tag['content'].replace("#", "") for tag in soup.find_all("meta", {"property" : "og:video:tag"})])

                print("is_family_friendly: ", is_family_friendly)
                print("video_id: ", video_id)
                print("author_id: ", author_id)
                print("yupload_time: ", yupload_time)
                print("keywords: ", keywords)

                # ytInitialData crawling
                yt_initial_data = driver.find_element_by_xpath(yt_initial_data_xpath)
                yt_initial_data_str = yt_initial_data.get_attribute("innerHTML").split(";")[0].replace('window["ytInitialData"] =', '').strip()
                # print("yt_initial_data: ", yt_initial_data_str)

                time.sleep(5)

                # data to json
                # json loading이 간헐적으로 안되는 현상 수정 필요
                yt_initial_data_json = json.loads(yt_initial_data_str)

                # print("yt_initial_data_json: ", yt_initial_data_json)

                # video_info_renderer
                video_info_renderer = yt_initial_data_json['contents']['twoColumnWatchNextResults']['results']['results']['contents']
                video_primary_info_renderer = video_info_renderer[0]['videoPrimaryInfoRenderer']
                video_secondary_info_renderer = video_info_renderer[1]['videoSecondaryInfoRenderer']

                # print("video_primary_info_renderer: ", video_primary_info_renderer)
                # print("video_secondary_info_renderer: ", video_secondary_info_renderer)

                # 좋아요 & 싫어요
                like_dislike_cnt = video_primary_info_renderer['sentimentBar']['sentimentBarRenderer']['tooltip']

                print("like_dislike_cnt: ", like_dislike_cnt)

                like_cnt = int(like_dislike_cnt.split(" / ")[0].replace(",", ""))
                dislike_cnt = int(like_dislike_cnt.split(" / ")[1].replace(",", ""))

                print("like_cnt: ", like_cnt, " type: ", type(like_cnt))
                print("dislike_cnt: ", dislike_cnt, " type: ", type(dislike_cnt))

                # 제목 타이틀 해시태그
                # 타이틀 해시태그가 없는 경우 빈문자열로 처리
                raw_title_hash_tags = video_primary_info_renderer.get('superTitleLink', "")
                if raw_title_hash_tags != "":
                    title_hash_tags = video_primary_info_renderer['superTitleLink']['runs']
                    title_hash_tag_list = []
                    for tag in title_hash_tags:
                        if tag['text'] not in [" ", ""]:
                            title_hash_tag_list.append(tag['text'][1:])
                            # print("title_hash_tag: ", tag['text'][1:])
                    title_hash_tags_str = ",".join(title_hash_tag_list)
                else:
                    title_hash_tags_str = ""
                print("title_hash_tags_str: ", title_hash_tags_str)

                # 내용 해시태그
                description_hash_tags = video_secondary_info_renderer['description']['runs']
                description_hash_tag_list = []
                for tag in description_hash_tags:
                    if tag['text'] not in [" ", ""]:
                        if "#" in tag['text']:
                            description_hash_tag_list.append(tag['text'][1:])
                            # print("description_hash_tag: ", tag['text'][1:])

                description_hash_tags_str = ",".join(description_hash_tag_list)
                print("description_hash_tags_str: ", description_hash_tags_str)

                # 구독자 수
                # 구독자 수가 없는 경우에 대한 예외처리(KeyError) -> 0 으로 처리
                raw_follow = video_secondary_info_renderer['owner']['videoOwnerRenderer'].get('subscriberCountText', "")
                if raw_follow != "":
                    raw_follow_cnt = video_secondary_info_renderer['owner']['videoOwnerRenderer']['subscriberCountText']['runs'][0]['text'][4:]
                    follow_cnt = raw_follow_cnt[:len(raw_follow_cnt)-1]
                    print("follow_cnt: ", follow_cnt)

                    # 구독자 수 수치화 필터링
                    num_count_dict = {
                        '천' : 1000,
                        '만' : 10000,
                        '억' : 100000000,
                        '조' : 1000000000000
                    }

                    # print("follow_cnt: ", follow_cnt)
                    # print("follow_num_cnt: ", follow_cnt[-1])
                    # 천, 만, 억, 조 에 걸리는 경우 -> 그 만큼 곱해서 리턴, 그게 아닐 경우 그냥 정수화 후 리턴
                    if follow_cnt[-1] in num_count_dict.keys():
                        follow_cnt_num = int(float(follow_cnt[:len(follow_cnt)-1]) * num_count_dict[follow_cnt[-1]])
                    else:
                        follow_cnt_num = int(follow_cnt)
                else:
                    follow_cnt_num = 0

                print("follow_cnt_num: ", follow_cnt_num, " type: ", type(follow_cnt_num))

                # 예외처리에 걸리지 않고 무사히 넘어왔다면 list에 데이터가 담긴다
                # scriptTag data appending
                title_list.append(title)
                author_list.append(author)
                description_list.append(description)
                url_list.append(url)
                thumb_url_list.append(thumb_url)
                view_cnt_list.append(view_cnt)
                duration_list.append(duration)
                youtube_category_list.append(youtube_category)
                youtube_category_sn_list.append(youtube_category_sn)

                # meta tag data appending
                is_family_friendly_list.append(is_family_friendly)
                video_id_list.append(video_id)
                author_id_list.append(author_id)
                yupload_time_list.append(yupload_time)
                keywords_list.append(keywords)

                # yInitialData data appending
                like_cnt_list.append(like_cnt)
                dislike_cnt_list.append(dislike_cnt)
                title_hash_tags_list.append(title_hash_tags_str)
                description_hash_tags_list.append(description_hash_tags_str)
                follow_cnt_list.append(follow_cnt_num)
                
            except Exception as e:
                print()
                print("Exception: ", e)
                driver.close()
                continue
            
            print("=" * 20, "Youtube Raw Data Descriptions Ended", "=" * 20, "\n")
            
            driver.close()

        # 얻어온 데이터를 dictionary에 저장
        youtube_raw_data['title'] = title_list
        youtube_raw_data['authorNick'] = author_list
        youtube_raw_data['description'] = description_list
        youtube_raw_data['urlPath'] = url_list
        youtube_raw_data['thumbUrl'] = thumb_url_list
        youtube_raw_data['likeCnt'] = like_cnt_list
        youtube_raw_data['dislikeCnt'] = dislike_cnt_list
        youtube_raw_data['followCnt'] = follow_cnt_list
        youtube_raw_data['viewCnt'] = view_cnt_list
        youtube_raw_data['duration'] = duration_list
        youtube_raw_data['utbCategory'] = youtube_category_list
        youtube_raw_data['utbCateSN'] = youtube_category_sn_list
        youtube_raw_data['isFamilyFriendly'] = is_family_friendly_list
        youtube_raw_data['titleHashTags'] = title_hash_tags_list
        youtube_raw_data['descHashTags'] = description_hash_tags_list
        youtube_raw_data['utbVideoID'] = video_id_list
        youtube_raw_data['authorID'] = author_id_list
        youtube_raw_data['yUploadDateTime'] = yupload_time_list
        youtube_raw_data['keywords'] = keywords_list
        
        print("youtube_raw_data: ", youtube_raw_data)
        
        # DataFrame 생성
        df = pd.DataFrame(youtube_raw_data)
        
        total_df.append(df, ignore_index = True)

        # 모아둔 DataFrame을 csv로 변환
        df.to_csv("./"+keyword+"_youtube_raw_data.csv", sep="|", index=False, encoding='utf-8-sig')

    return total_df

In [28]:
scrap_search_detail_video_data(youtube_id_dict)

==========outer loop==========
keyword:  한국 프로야구
video_id_list:  ['doorlUyE9pI', 'L0Y38pQbEeQ', 'VjwJSj0AW4g', 'A5vCPtjOe8Q', 'Zeja7fC1Okc', '7ARbpqOgHM8', 'WC2MB_lZ0HQ', 'ugsFwK74KDU', 'FeqhJVzlE-4', 'B-ZFW6-FBJ8', 'kH-ht4CyUKg', 'kQxq3PNE0gw', 'leBGyyECGPk', '7eYu507yrQ8', '-P68flOonvE', 'FCEPBiUPPho', 'pp0a2SbPwRA', 'VFufiK95fGU', 'De58PXoH4IU', 'wVo7-0zDNps']
==========before inner loop==========
==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  [11월 17일] 두산 vs NC 한국시리즈 1차전 경기 하이라이트 | 프로 야구 2020 NC Dinos vs Doosan Bears Highlights
author:  100야구큰 팬
description:  #두산vsNC, #NCvs두산,

[11월 17일] 두산 vs NC 한국시리즈 1차전 경기 하이라이트 | 프로 야구 2020 NC Dinos vs Doosan Bears Highlights
thumb_url:  https://i.ytimg.com/vi/doorlUyE9pI/hqdefault.jpg
view_cnt:  23382  type:  <class 'int'>
duration:  525  type:  <class 'int'>
youtube_category:  People & Blogs
youtube_category_id:  22
is_family_friendly:  True
video_id:  doorlUyE9pI
author_id:  UC


==================== Youtube Raw Data Descriptions ====================
title:  KBO 프로야구 승부조작 투구모음(NC 이태양, 유창식)
author:  온리원이글스
description:  2016 한국 프로야구 최악의 사건

korea baseball pitcher match fixing scandal
thumb_url:  https://i.ytimg.com/vi/B-ZFW6-FBJ8/maxresdefault.jpg
view_cnt:  3905236  type:  <class 'int'>
duration:  361  type:  <class 'int'>
youtube_category:  Sports
youtube_category_id:  17
is_family_friendly:  True
video_id:  B-ZFW6-FBJ8
author_id:  UCgA_bbXQvYHXGQhOQEzPEkg
yupload_time:  2016-10-17
keywords:  KBO,프로야구,승부조작,NC 이태양,이태양,유창식,match fixing,baseball scandal
like_dislike_cnt:  5,637 / 875
like_cnt:  5637  type:  <class 'int'>
dislike_cnt:  875  type:  <class 'int'>
title_hash_tags_str:  
description_hash_tags_str:  
follow_cnt:  1.83만
follow_cnt_num:  18300  type:  <class 'int'>
==================== Youtube Raw Data Descriptions Ended ==================== 

==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title: 

like_dislike_cnt:  968 / 41
like_cnt:  968  type:  <class 'int'>
dislike_cnt:  41  type:  <class 'int'>
title_hash_tags_str:  
description_hash_tags_str:  
follow_cnt:  1.23만
follow_cnt_num:  12300  type:  <class 'int'>
==================== Youtube Raw Data Descriptions Ended ==================== 

==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  KBO 프로야구 다시는 나오기 힘든 기록들
author:  야구박사
description:  LEGEND
thumb_url:  https://i.ytimg.com/vi/wVo7-0zDNps/hqdefault.jpg
view_cnt:  840048  type:  <class 'int'>
duration:  541  type:  <class 'int'>
youtube_category:  Sports
youtube_category_id:  17
is_family_friendly:  True
video_id:  wVo7-0zDNps
author_id:  UC_xgQWu3RvW58p0_cofgfmA
yupload_time:  2020-03-04
keywords:  프로야구 KBO 기록

Exception:  Unterminated string starting at: line 1 column 16736 (char 16735)
youtube_raw_data:  {'title': ['[11월 17일] 두산 vs NC 한국시리즈 1차전 경기 하이라이트 | 프로 야구 2020 NC Dinos vs Doosan Bears Highlights', '한국 프로

like_dislike_cnt:  5,835 / 254
like_cnt:  5835  type:  <class 'int'>
dislike_cnt:  254  type:  <class 'int'>
title_hash_tags_str:  
description_hash_tags_str:  
follow_cnt:  143만
follow_cnt_num:  1430000  type:  <class 'int'>
==================== Youtube Raw Data Descriptions Ended ==================== 

==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  [20/21 UCL] 유벤투스 vs 바르셀로나 H/L
author:  SPOTV
description:  유벤투스 vs 바르셀로나 경기 하이라이트
thumb_url:  https://i.ytimg.com/vi/DWwbxsVwPEs/maxresdefault.jpg
view_cnt:  1146776  type:  <class 'int'>
duration:  705  type:  <class 'int'>
youtube_category:  Sports
youtube_category_id:  17
is_family_friendly:  True
video_id:  DWwbxsVwPEs
author_id:  UCtm_QoN2SIxwCE-59shX7Qg
yupload_time:  2020-10-28
keywords:  SPOTV,스포티비,메시,챔피언스리그,바르샤,호날두,디발라
like_dislike_cnt:  6,861 / 294
like_cnt:  6861  type:  <class 'int'>
dislike_cnt:  294  type:  <class 'int'>
title_hash_tags_str:  
description_hash_t


==================== Youtube Raw Data Descriptions ====================
title:  11/19 「스포츠토토」프로토,토토분석,축구분석,해외축구분석,베트맨토토
author:  구구스포츠
description:  #스포츠토토 #프로토 #토토
thumb_url:  https://i.ytimg.com/vi/VpEre3ksO5w/maxresdefault.jpg
view_cnt:  1567  type:  <class 'int'>
duration:  450  type:  <class 'int'>
youtube_category:  People & Blogs
youtube_category_id:  22
is_family_friendly:  True
video_id:  VpEre3ksO5w
author_id:  UCTMLxQpAVJK4f53bKjIz1hw
yupload_time:  2020-11-16
keywords:  스포츠토토,프로토,배트맨토토,베트맨토토,프로토분석,축구분석,야구분석,농구분석,토토분석
like_dislike_cnt:  65 / 0
like_cnt:  65  type:  <class 'int'>
dislike_cnt:  0  type:  <class 'int'>
title_hash_tags_str:  스포츠토토,프로토,토토
description_hash_tags_str:  스포츠토토,프로토,토토
follow_cnt:  5.41천
follow_cnt_num:  5410  type:  <class 'int'>
==================== Youtube Raw Data Descriptions Ended ==================== 

==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  현재 손흥민 경기중 나온 장면을 해외에서 크게 놀라워 하는 

...


==================== Youtube Raw Data Descriptions ====================
title:  상어는 배꼽이 없어요 | 상어동화 | 창작동화 | 깨비키즈 KEBIKIDS
author:  깨비키즈 [KEBIKIDS]
description:  아기상어 조슈아가 아기 고래의 배꼽을 발견했어요. 하지만 아무리 찾아도 자신의 몸에서는 배꼽을 발견할 수 없었어요. 엄마랑 연결되었던 소중한 흔적인 배꼽이 왜 없을까요?
★ 깨비키즈 구독하기 ►► https://www.youtube.com/channel/UCD_-N4vf-tsqbUxrQ4CrJIA/?sub_confirmation=1

-----

대한민국 No.1 유아교육 깨비키즈에서 더 많은 영상을 만나보세요! 
★ 깨비키즈 : http://www.kebikids.com 

Copyright © KEBIKIDS Corp. All Rights Reserved.
thumb_url:  https://i.ytimg.com/vi/zxWlDeaq2vY/maxresdefault.jpg
view_cnt:  1526386  type:  <class 'int'>
duration:  256  type:  <class 'int'>
youtube_category:  Education
youtube_category_id:  27
is_family_friendly:  True
video_id:  zxWlDeaq2vY
author_id:  UCD_-N4vf-tsqbUxrQ4CrJIA
yupload_time:  2020-11-05
keywords:  아기상어,상어가족,상어이야기,어린이동화,상어동화,상어가족소풍,가족소풍,깨비키즈,깨비,깨비티비,kebikids,깨비동화,아가동화,아기동화,바다동화,창작동화,친구놀이,상어친구,아기상어 친구,상어는 배꼽이 없어요,상어배꼽,배꼽
like_dislike_cnt:  6,074 / 3,129
like_cnt:  6074  type:  <class 'int'>
dislik


==================== Youtube Raw Data Descriptions ====================
title:  수영장에서 상어가 짠!! 탐사선을 타고 바다속 친구들을 구해요! 레고시티 해양 장난감 상황극 놀이 Pretend Play Shark in Pool [제이제이 튜브-JJ tube]
author:  제이제이 튜브 [JJ Tube]
description:  ♥JJ튜브 무료구독 : https://goo.gl/6wkPci ♥
제이제이패밀리가 수영장에서 상어를 만났어요. 레고 시티 해양 시리즈로 구해줘요
#제이제이튜브 #레고시티 #레고해양 #상황극
레고 시티 영상 URL: https://youtu.be/sKWVP0IF-u4
- 이 영상은 레고 코리아로부터 제품을 지원받았습니다.

[제이제이 튜브 - JJ tube] 
제이제이 튜브를 구독하시면 무료로
재미있고 신기한 영상을 볼수 있어요~
많이 많이 구독해주시고
영상이 재미있으셨으면 좋아요 꾹 눌러주세요 ^^

채널 구독(Subscribe) : https://goo.gl/6wkPci
캐스트(NAVER TVcast) : http://tvcast.naver.com/jjtube
카카오 스토리(Kakao Story) : https://story.kakao.com/jjtube


[최신 꿀잼 동영상]
불이야 불! 브루더 소방차로 불을 꺼요. 소방차 크레인 레미콘 중장비 장난감 이름 공부 편백나무 큐브 놀이 bruder toy for kids
https://youtu.be/6ieXr7w6fjE

알파벳 퍼즐매트로 글자 맞추며 영어 공부해요. 알파벳송 인기동요 장난감 놀이 ABC Song Nursery Rhymes for children
https://youtu.be/kni19fNHEPE

티라노 공룡뼈가 살아났다! 공룡 화석 쥬라기 장난감 놀이 Jurassic World Dainosaur TREX skeleton Toy Play
https://youtu.be/y-Cbhn6vxRs

마블 피젯


Exception:  'sentimentBar'
==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  백상아리가 '식인 상어'? 알고 보니 45년간 억울하게 뒤집어 쓴 누명이었다!
author:  엠빅뉴스
description:  가장 큰 포식성 어류, 백상아리의 '식인 상어'란 악명은 45년간 억울하게 뒤집어 쓴 '누명'이라고 호주 과학자들이 밝혔습니다. 주로 해저에서 지내며 해수면으로는 올라올 시간도 없다는데 어떻게 누명을 썼을까요?

#상어 #백상아리 #죠스 #식인상어 #누명
thumb_url:  https://i.ytimg.com/vi/s_3vVYHoilI/maxresdefault.jpg
view_cnt:  296916  type:  <class 'int'>
duration:  314  type:  <class 'int'>
youtube_category:  News & Politics
youtube_category_id:  25
is_family_friendly:  True
video_id:  s_3vVYHoilI
author_id:  UCYRrUD5v5j_Ei8sKOo7mhoQ
yupload_time:  2020-06-10
keywords:  

Exception:  'videoPrimaryInfoRenderer'
==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  원시 상어의 모습을 간직한 심해의 희귀상어들
author:  내셔널지오그래픽 - National Geographic Korea
description:  외딴 해저에 보기 드문 희귀 상어들이 살고 있습니다.. 
최소 수명 400 살의 가장 수명이 긴 척추동물 그린란드상어와 
약 9,500 만년 동안 

like_dislike_cnt:  15,682 / 9,397
like_cnt:  15682  type:  <class 'int'>
dislike_cnt:  9397  type:  <class 'int'>
title_hash_tags_str:  바다탐험대옥토넛,Octonauts
description_hash_tags_str:  바다탐험대옥토넛,Octonauts,바다탐험대옥토넛,Octonauts
follow_cnt:  25만
follow_cnt_num:  250000  type:  <class 'int'>
==================== Youtube Raw Data Descriptions Ended ==================== 

==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  바닷속 상어 그리기 / 고래 그리기 Drawing shark and whale 바다생물 그리기 상어모음 상어가족
author:  마미튜브 [Mommy Tube]
description:  백상아리 / 범고래 / 벨루가 / 환도상어 / 돌묵상어 / 고래상어 / 일각돌고래 / 톱상어 / 수염상어 까지
바닷속 상어 친구들과 고래를 그려보았어요
무섭지만 그림으로 그리는 상어는 귀엽네요 :)
thumb_url:  https://i.ytimg.com/vi/LzLvCAR6M1w/maxresdefault.jpg
view_cnt:  65779  type:  <class 'int'>
duration:  283  type:  <class 'int'>
youtube_category:  Pets & Animals
youtube_category_id:  15
is_family_friendly:  True
video_id:  LzLvCAR6M1w
author_id:  UCBhj5r1-b6_5MGOVDllaH1A
yupload_time:  2020-03-

,title,authorNick,description,urlPath,thumbUrl,likeCnt,dislikeCnt,followCnt,viewCnt,duration,utbCategory,utbCateSN,isFamilyFriendly,titleHashTags,descHashTags,utbVideoID,authorID,yUploadDateTime,keywords


In [41]:
# csv file 불필요한 0번째 컬럼 제거(test)
df = pd.read_csv("./공룡_youtube_raw_data2.csv", index_col=False)
df = df.drop("Unnamed: 0", axis=1)

df.to_csv("./공룡_youtube_raw_data2"+"_fix"+".csv", index=False, encoding='utf-8-sig')

In [42]:
# 기존에 default index_column을 추가한 csv 파일을 불러온 후 index_column 제거 후 다시 저장
for keyword in all_search_keyword_list:
    df = pd.read_csv("./"+keyword+"_youtube_raw_data.csv", index_col=False)
    df = df.drop("Unnamed: 0", axis=1)
    
    df.to_csv("./"+keyword+"_youtube_raw_data"+"_fix"+".csv", index=False, encoding='utf-8-sig')

In [60]:
# 기존 데이터에서 문자열 데이터 중 "," -> "|"로 변경 및 문자열 내부의 \"를 ""로 변경
df = pd.read_csv("./공룡_youtube_raw_data2"+"_fix"+".csv", index_col=False, encoding='utf-8-sig')

# 문자열 정리(filtering)
# 문자열을 필터링하는 함수 정의
def filter_string_data(data):
    data = data.replace(",", "|").replace('"', '')
    
    return data

df['title'] = df['title'].apply(filter_string_data)
df['authorNick'] = df['authorNick'].apply(filter_string_data)
df['description'] = df['description'].apply(filter_string_data)

# df
df.to_csv("./공룡_youtube_raw_data2"+"_fix2"+".csv", sep="|", index=False, encoding='utf-8-sig')

In [55]:
df2 = pd.read_csv("./공룡_youtube_raw_data2"+"_fix2"+".csv", sep='|', index_col=False, encoding='utf-8-sig')
df2

,title,authorNick,description,urlPath,thumbUrl,likeCnt,dislikeCnt,followCnt,viewCnt,duration,utbCategory,utbCateSN,isFamilyFriendly,titleHashTags,descHashTags,utbVideoID,authorID,yUploadDateTime,keywords
0,5 Brachiosaurus Dinosaurs! Brichiosaurus Dino ...,Boys Playtime,5 Brachiosaurus Dinosaurs! Brichiosaurus Dino ...,https://www.youtube.com/watch?v=JLFtlXPuFaw,https://i.ytimg.com/vi/JLFtlXPuFaw/maxresdefau...,585,256,1240000,100568,333,Entertainment,24,True,"dinosaurs,brachiosaurus,브라키오사우루스","dinosaurs,brachiosaurus,브라키오사우루스",JLFtlXPuFaw,UCCrGfO9A8eLdSsB4bWbMhTg,2020-11-16,"dinosaur heads,brachiosaurus head,dinosaur egg..."
1,공룡 동화 모음집 | 공룡들을 동화로 만나요 | 스피노사우르스 VS 티라노사우르스 ...,핑크퐁 (인기 동요・동화),★호기 공식 한글 유튜브 채널 오픈★ 호기 유튜버 전격 데뷔! \n지금 바로 호기 ...,https://www.youtube.com/watch?v=7IadsIPjO9I,https://i.ytimg.com/vi/7IadsIPjO9I/maxresdefau...,2673,1951,8750000,1244668,1502,Education,27,True,"핑크퐁,공룡동화,집에서함께해요","핑크퐁,공룡동화,집에서함께해요",7IadsIPjO9I,UCqXwKu6dKobXEQFhdKtiJLQ,2020-04-18,"핑크퐁,pinkfong,스마트스터디,smartstudy,보들북,삼성출판사,가족,유아..."
2,공룡 동요 총집합! | 티라노사우루스| 안킬로사우루스| 트리케라톱스 외 22곡 | ...,핑크퐁 (인기 동요・동화),★호기 공식 한글 유튜브 채널 오픈★ 호기 유튜버 전격 데뷔! \n지금 바로 호기 ...,https://www.youtube.com/watch?v=v2tsHXQI6d0,https://i.ytimg.com/vi/v2tsHXQI6d0/maxresdefau...,30675,18561,8750000,17214980,1778,Education,27,True,NaN,NaN,v2tsHXQI6d0,UCqXwKu6dKobXEQFhdKtiJLQ,2017-11-25,"핑크퐁,pinkfong,스마트스터디,smartstudy,보들북,삼성출판사,가족,유아..."
3,쥬라기월드 에볼루션 티렉스 공룡 배틀 인도랩터 인도미누스 렉스 트리케라톱스,Toy Dino TV,쥬라기월드 에볼루션 공룡 배틀 인도랩터| 티렉스| 인도미누스 렉스| 트리케라톱스| ...,https://www.youtube.com/watch?v=p_Fj6KmgXX4,https://i.ytimg.com/vi/p_Fj6KmgXX4/maxresdefau...,404,148,1140000,100154,492,Gaming,20,True,NaN,NaN,p_Fj6KmgXX4,UCy7PtHS0_bX8kiSddrjrI5A,2020-10-24,"Toy Dino TV,Toy Dinosaur,Jurassic World,恐竜,Jur..."
4,목이 긴 공룡 & 티렉스 쥬라기월드 에볼루션 브라키오사우루스 아파토사우루스 디플로도...,Toy Dino TV,공룡 티렉스 & 목이 긴 공룡 쥬라기월드 에볼루션 슐라이히 파포 컬렉타 공룡피규어 ...,https://www.youtube.com/watch?v=IRZF0IZ6BhI,https://i.ytimg.com/vi/IRZF0IZ6BhI/maxresdefau...,1105,418,1140000,248188,624,Entertainment,24,True,NaN,NaN,IRZF0IZ6BhI,UCy7PtHS0_bX8kiSddrjrI5A,2020-10-20,"Toy Dino TV,Toy Dinosaur,Jurassic World,恐竜,Jur..."
5,공룡동요 총집합! | 우리아이 좋아하는 공룡동요 모아듣기! | 티라노사우루스 | 초...,티디키즈 by 지니키즈,#인기동요 #어린이동요 #묶음동요\r\n\r\n티라노사우루스송\n공룡왕\n티라노 가...,https://www.youtube.com/watch?v=vvAly7LI23U,https://i.ytimg.com/vi/vvAly7LI23U/maxresdefau...,107,66,1140000,27620,1826,Education,27,True,"인기동요,어린이동요,묶음동요","인기동요,어린이동요,묶음동요,티디키즈,티디동요,티디인기동요,tidikids,tidi...",vvAly7LI23U,UCqvhycZ4nzxTHU4RxHjYgWQ,2020-11-10,"티디,티디키즈,티디동요,티디인기동요,인기동요,유아동요,어린이동요,율동동요,신나는동요..."
6,[깨미 탐험대] - 46화 최고로 무서운 공룡은 (EBSXGNG),EBS 키즈,곰퉁이가 읽고 있는 동화책이 무언지 궁금한 깨미와코요코요. 들여다보니 귀여운 아기공...,https://www.youtube.com/watch?v=JAYa0ffVVQY,https://i.ytimg.com/vi/JAYa0ffVVQY/maxresdefau...,4784,3481,3250000,1518261,692,Film & Animation,1,True,NaN,NaN,JAYa0ffVVQY,UCx_dzPI6efSohgtOkAG43-A,2019-06-17,"EBS,EBSkids,EBS 유아,어린이,유아,교육,애니메이션,교육애니메이션,교육만..."
7,서프라이즈!! 변신 공룡 계란 칼라 놀이! 레드| 블루| 그린!! | 두두팝토이,두두팝토이,- 구독하기 클릭! 👉 http://bit.ly/2kDwZyo\n- Subscrib...,https://www.youtube.com/watch?v=m40Dt_TZFBI,https://i.ytimg.com/vi/m40Dt_TZFBI/maxresdefau...,1565,490,1020000,382390,450,Film & Animation,1,True,"장난감,애니메이션,두두팝토이","장난감,애니메이션,두두팝토이",m40Dt_TZFBI,UC7AmWE0eqBymIWwRnk8vmqg,2020-06-20,"dudupoptoy,장난감,애니메이션,animation,toys,toy"
8,🌟 고고다이노 공룡탐험대 시즌5 7화 🌟 | 움직이는 바위 스테고사우루스 | 고고다...,고고다이노- GOGO DINO| 고고 티비| GoGoTV,#고고다이노 #공룡탐험대 #시즌5\n\n안녕! 친구들!\n오늘은 딱딱한 등을 가지고...,https://www.youtube.com/watch?v=IUAjhm7oE9c,https://i.ytimg.com/vi/IUAjhm7oE9c/maxresdefau...,775,474,741000,258295,1839,Film & Animation,1,True,"고고다이노,공룡탐험대,시즌5","고고다이노,공룡탐험대,시즌5",IUAjhm7oE9c,UCySazY_iw_LlE6WHyQXuZ6g,2020-10-02,"공룡,애니메이션,만화,다이노,고고다이노,유아용,애니,교육,어린이,GoGoDino,공..."
9,혼자 집에 있으면 생기는 일,공룡,몰래 먹으니 더 맛있다\n\n\n#공룡\n#불편한동거\n---------------...,https://www.youtube.com/watch?v=-FqABXx2t4o,https://i.ytimg.com/vi/-FqABXx2t4o/maxresdefau...,9942,131,342000,417437,1241,Gaming,20,True,"공룡,불편한동거","공룡,불편한동거",-FqABXx2t4o,UCWf4QKgnTyvmehcRqw-JldQ,2020-11-12,"마인크래프트,공룡,각별,마인애플,카운터,꾸몽,마플,운터,공룡티비,공룡

In [ ]:
# 데이터 단일 크롤링 테스트
def scrap_test_detail_video_data(video_id='y57Zldv2bik'):
    
    # webdriver 초기화
    driver_path = './chromedriver'
    driver = webdriver.Chrome(driver_path)
    driver.implicitly_wait(30) # or bigger second
    
    # origin_url 정의
    origin_url = 'https://www.youtube.com/watch?v='
    
    # 테스트용 단일 youtube_id 정의
    # (임시) 딘일로 넣어주는 값
    # video_id = 'y57Zldv2bik'
    
    # youtube_detail_raw_data를 받을 dictionary 정의
    youtube_raw_data = {}
    
    # 넣어주는 url 값
    url = origin_url + video_id
    
    # scriptTag에서 긁어지는 데이터
    script_tag_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[1]/ytd-player-microformat-renderer/script'
    
    # meta tag에서 긁어지는 데이터 -> soup.find, soup.findAll 이용해서 긁기
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'lxml')
    
    # ytInitialData에서 긁어지는 데이터 -> 복잡한 계층의 json 데이터 기반
    yt_initial_data_xpath = '/html/body/script[19]'
    
    # 수집할 데이터의 리스트
    title_list = []
    author_list = []
    description_list = []
    url_list = []
    thumb_url_list = []
    like_cnt_list = []
    dislike_cnt_list = []
    follow_cnt_list = []
    view_cnt_list = []
    duration_list = []
    youtube_category_list = []
    youtube_category_sn_list = []
    is_family_friendly_list = []
    title_hash_tags_list = []
    description_hash_tags_list = []
    video_id_list = [] 
    author_id_list = []
    yupload_time_list = []
    keywords_list = []
    
    driver.get(url)
    time.sleep(5)
    
    # Data 크롤링
    # 각 유형별로 다른 로직으로 데이터를 크롤링한다
    
    # script tag id=scriptTag의 내부 HTML 내용을 가져온다(scriptTag data crawling)
    script_tag_str = driver.find_element_by_xpath(script_tag_xpath).get_attribute("innerHTML")

    script_tag_json = json.loads(script_tag_str)
    
    time.sleep(5)

    # print("script_tag_json: ", script_tag_json)
    # print("script_tag_json_type: ", type(script_tag_json))

    # Data result test
    title = script_tag_json['name']
    author = script_tag_json['author']
    description = script_tag_json['description']
    thumb_url = script_tag_json['thumbnailUrl'][0]
    view_cnt = int(script_tag_json['interactionCount'])
    duration = int(script_tag_json['duration'][2:len(script_tag_json['duration'])-1].replace(",", ""))
    youtube_category = script_tag_json['genre']
    # addition : youtube_category_sn
    youtube_category_sn = youtube_category_dict[youtube_category]

    print("title: ", title)
    print("author: ", author)
    print("description: ", description)
    print("thumb_url: ", thumb_url)
    print("view_cnt: ", view_cnt, " type: ", type(view_cnt))
    print("duration: ", duration, " type: ", type(duration))
    print("youtube_category: ", youtube_category)
    print("youtube_category_id: ", youtube_category_sn)
    
    # scriptTag data appending
    title_list.append(title)
    author_list.append(author)
    description_list.append(description)
    url_list.append(url)
    thumb_url_list.append(thumb_url)
    view_cnt_list.append(view_cnt)
    duration_list.append(duration)
    youtube_category_list.append(youtube_category)
    youtube_category_sn_list.append(youtube_category_sn)
    
    
    # meta tag data crawling
    is_family_friendly = soup.find("meta", {"itemprop" : "isFamilyFriendly"})['content']
    author_id = soup.find("meta", {"itemprop" : "channelId"})['content']
    yupload_time = soup.find("meta", {"itemprop" : "uploadDate"})['content']
    keywords = ",".join([tag['content'] for tag in soup.find_all("meta", {"property" : "og:video:tag"})])

    print("is_family_friendly: ", is_family_friendly)
    print("video_id: ", video_id)
    print("author_id: ", author_id)
    print("yupload_time: ", yupload_time)
    print("keywords: ", keywords)
    
    # meta tag data appending
    is_family_friendly_list.append(is_family_friendly)
    video_id_list.append(video_id)
    author_id_list.append(author_id)
    yupload_time_list.append(yupload_time)
    keywords_list.append(keywords)
    
    # ytInitialData crawling
    yt_initial_data = driver.find_element_by_xpath(yt_initial_data_xpath)
    yt_initial_data_str = yt_initial_data.get_attribute("innerHTML").split(";")[0].replace('window["ytInitialData"] =', '').strip()
    # print("yt_initial_data: ", yt_initial_data_str)

    time.sleep(5)

    # data to json
    # json loading이 간헐적으로 안되는 현상 수정 필요
    yt_initial_data_json = json.loads(yt_initial_data_str)

    # print("yt_initial_data_json: ", yt_initial_data_json)

    # video_info_renderer
    video_info_renderer = yt_initial_data_json['contents']['twoColumnWatchNextResults']['results']['results']['contents']
    video_primary_info_renderer = video_info_renderer[0]['videoPrimaryInfoRenderer']
    video_secondary_info_renderer = video_info_renderer[1]['videoSecondaryInfoRenderer']

    # print("video_primary_info_renderer: ", video_primary_info_renderer)
    # print("video_secondary_info_renderer: ", video_secondary_info_renderer)

    # 좋아요 & 싫어요
    like_dislike_cnt = video_primary_info_renderer['sentimentBar']['sentimentBarRenderer']['tooltip']

    print("like_dislike_cnt: ", like_dislike_cnt)
    
    like_cnt = int(like_dislike_cnt.split(" / ")[0].replace(",", ""))
    dislike_cnt = int(like_dislike_cnt.split(" / ")[1].replace(",", ""))
    
    print("like_cnt: ", like_cnt, " type: ", type(like_cnt))
    print("dislike_cnt: ", dislike_cnt, " type: ", type(dislike_cnt))

    # 제목 타이틀 해시태그
    raw_title_hash_tags = video_primary_info_renderer.get('superTitleLink', "")
    if raw_title_hash_tags != "":
        title_hash_tags = video_primary_info_renderer['superTitleLink']['runs']
        title_hash_tag_list = []
        for tag in title_hash_tags:
            if tag['text'] not in [" ", ""]:
                title_hash_tag_list.append(tag['text'][1:])
                # print("title_hash_tag: ", tag['text'][1:])
        title_hash_tags_str = ",".join(title_hash_tag_list)
    else:
        title_hash_tags_str = raw_title_hash_tags
    print("title_hash_tags_str: ", title_hash_tags_str)

    # 내용 해시태그
    description_hash_tags = video_secondary_info_renderer['description']['runs']
    description_hash_tag_list = []
    for tag in description_hash_tags:
        if tag['text'] not in [" ", ""]:
            if "#" in tag['text']:
                description_hash_tag_list.append(tag['text'][1:])
                # print("description_hash_tag: ", tag['text'][1:])

    description_hash_tags_str = ",".join(description_hash_tag_list)
    print("description_hash_tags_str: ", description_hash_tags_str)

    # 구독자 수
    # 구독자 수가 없는 경우에 대한 예외처리(KeyError) -> 0 으로 처리
    raw_follow = video_secondary_info_renderer['owner']['videoOwnerRenderer'].get('subscriberCountText', "")
    if raw_follow != "":
        raw_follow_cnt = video_secondary_info_renderer['owner']['videoOwnerRenderer']['subscriberCountText']['runs'][0]['text'][4:]
        follow_cnt = raw_follow_cnt[:len(raw_follow_cnt)-1]
        print("follow_cnt: ", follow_cnt)

        # 구독자 수 수치화 필터링
        num_count_dict = {
            '천' : 1000,
            '만' : 10000,
            '억' : 100000000,
            '조' : 1000000000000
        }

        # print("follow_cnt: ", follow_cnt)
        # print("follow_num_cnt: ", follow_cnt[-1])
        # 천, 만, 억, 조 에 걸리는 경우 -> 그 만큼 곱해서 리턴, 그게 아닐 경우 그냥 정수화 후 리턴
        if follow_cnt[-1] in num_count_dict.keys():
            follow_cnt_num = int(float(follow_cnt[:len(follow_cnt)-1]) * num_count_dict[follow_cnt[-1]])
        else:
            follow_cnt_num = int(follow_cnt)
    else:
        follow_cnt_num = 0
    print("follow_cnt_num: ", follow_cnt_num, " type: ", type(follow_cnt_num))
    
    # yInitialData data appending
    like_cnt_list.append(like_cnt)
    dislike_cnt_list.append(dislike_cnt)
    title_hash_tags_list.append(title_hash_tags_str)
    description_hash_tags_list.append(description_hash_tags_str)
    follow_cnt_list.append(follow_cnt_num)
    
    driver.close()
    
    # 얻어온 데이터를 dictionary에 저장
    youtube_raw_data['title'] = title_list
    youtube_raw_data['authorNick'] = author_list
    youtube_raw_data['description'] = description_list
    youtube_raw_data['urlPath'] = url_list
    youtube_raw_data['thumbUrl'] = thumb_url_list
    youtube_raw_data['likeCnt'] = like_cnt_list
    youtube_raw_data['dislikeCnt'] = dislike_cnt_list
    youtube_raw_data['followCnt'] = follow_cnt_list
    youtube_raw_data['viewCnt'] = view_cnt_list
    youtube_raw_data['duration'] = duration_list
    youtube_raw_data['utbCategory'] = youtube_category_list
    youtube_raw_data['utbCateSN'] = youtube_category_sn_list
    youtube_raw_data['isFamilyFriendly'] = is_family_friendly_list
    youtube_raw_data['titleHashTags'] = title_hash_tags_list
    youtube_raw_data['descHashTags'] = description_hash_tags_list
    youtube_raw_data['utbVideoID'] = video_id_list
    youtube_raw_data['authorID'] = author_id_list
    youtube_raw_data['yUploadDateTime'] = yupload_time_list
    youtube_raw_data['keywords'] = keywords_list
    
    # DataFrame 생성
    df = pd.DataFrame(youtube_raw_data)
    
    return df

In [ ]:
# 단일 크롤링 테스트 결과
# scrap_test_detail_video_data(video_id='VjwJSj0AW4g')

In [ ]:
# scrap_test_detail_video_data(video_id='doorlUyE9pI')

In [ ]:
# scrap_test_detail_video_data(video_id='L0Y38pQbEeQ')

In [ ]:
# scrap_test_detail_video_data(video_id='7eYu507yrQ8')